### Notes

* changed shape[1] from 4 to 6

In [3]:
%matplotlib inline

In [4]:
import sys
import numpy as np
import tensorflow as tf

from batcher import Batcher
from metrics import acc012

In [5]:
X_train = np.load('../../data/bin/train/deal.npy')
y_train = np.load('../../data/bin/train/tricks_spades.npy')

X_val = np.load('../../data/bin/val/deal.npy')
y_val = np.load('../../data/bin/val/tricks_spades.npy')

n_examples = X_train.shape[0]

n_h = X_train.shape[1]
n_w = X_train.shape[2]
n_c = X_train.shape[3]

batch_size = 64
n_iterations = 2000000
display_step = 1000

learning_rate = 0.001

n_hidden_units = 128

l2_reg = 0.05

In [6]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((10800000, 4, 13, 4), (100000, 4, 13, 4), (10800000, 1), (100000, 1))

In [7]:
X = tf.placeholder(tf.float32, shape=[None, n_h, n_w, n_c])
Y = tf.placeholder(tf.float32, shape=[1, None])
strides = [1,1,1,1]

conv1_w = tf.get_variable('c1w', shape=[1, 6, 4, 32], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv1_z = tf.nn.conv2d(X, filter=conv1_w, strides=strides, padding='SAME')
conv1_a = tf.nn.relu(conv1_z)

conv2_w = tf.get_variable('c2w', shape=[1, 6, 32, 64], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv2_z = tf.nn.conv2d(conv1_a, filter=conv2_w, strides=strides, padding='SAME')
conv2_a = tf.nn.relu(conv2_z)

conv3_w = tf.get_variable('c3w', shape=[1, 6, 64, 128], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv3_z = tf.nn.conv2d(conv2_a, filter=conv3_w, strides=strides, padding='SAME')
conv3_a = tf.nn.relu(conv3_z)

conv4_w = tf.get_variable('c4w', shape=[4, 6, 128, 512], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv4_z = tf.nn.conv2d(conv3_a, filter=conv4_w, strides=strides, padding='VALID')
conv4_a = tf.nn.relu(conv4_z)

fc_in = tf.contrib.layers.flatten(conv4_a)
fc_w = tf.get_variable('fcw', shape=[n_hidden_units, fc_in.shape.as_list()[1]], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
fc_b = tf.Variable(np.zeros((n_hidden_units, 1)), dtype=tf.float32)
fc_z = tf.add(tf.matmul(fc_w, tf.transpose(fc_in)), fc_b)
fc_a = tf.nn.relu(fc_z)

w_out = tf.get_variable('w_out', shape=[1, n_hidden_units], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
b_out = tf.Variable(np.zeros((1, 1)), dtype=tf.float32)
pred = tf.add(tf.matmul(w_out, fc_a), b_out)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [8]:
cost = tf.reduce_mean(tf.squared_difference(pred, Y))

In [9]:
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9).minimize(cost)

In [10]:
init = tf.global_variables_initializer()

In [11]:
batch = Batcher(n_examples, batch_size)
cost_train_batch = Batcher(n_examples, 10000)
cost_val_batch = Batcher(100000, 10000)

In [ ]:
costs = []

with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = batch.next_batch([X_train, y_train])
        
        if iteration % display_step == 0:
            sys.stdout.write('*')
            x_batch_c, y_batch_c = cost_train_batch.next_batch([X_train, y_train])
            x_batch_v, y_batch_v = cost_val_batch.next_batch([X_val, y_val])
            c = sess.run(cost, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            costs.append(c)
            pred_train = sess.run(pred, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            pred_val = sess.run(pred, feed_dict={X: x_batch_v, Y: y_batch_v.T})
            print('it={} cost={}'.format(iteration, c))
            print(acc012(y_batch_c, pred_train.T))
            print(acc012(y_batch_v, pred_val.T))
            
        sess.run(optimizer, feed_dict={X:x_batch, Y:y_batch.T})
    
    # Testing
    graph = tf.get_default_graph()
    
    X_test = np.load('../../data/bin/test/deal.npy')
    y_test = np.load('../../data/bin/test/tricks_spades.npy')
    
    print('\n\nTesting')
    predictions = sess.run(pred, feed_dict = {X:X_test})

    print(acc012(y_test.ravel(), predictions.round().ravel()))

*it=0 cost=47.15970230102539
(0.0112, 0.042, 0.0964)
(0.0133, 0.0412, 0.0958)
*it=1000 cost=1.0512747764587402
(0.3919, 0.8632, 0.9806)
(0.3872, 0.8632, 0.9813)
*it=2000 cost=1.078285574913025
(0.3778, 0.8561, 0.9812)
(0.385, 0.8586, 0.984)
*it=3000 cost=0.8676970601081848
(0.4149, 0.8954, 0.9906)
(0.42, 0.8992, 0.9925)
*it=4000 cost=0.766218900680542
(0.446, 0.9126, 0.9933)
(0.4423, 0.9161, 0.9941)
*it=5000 cost=0.6832202076911926
(0.4666, 0.9329, 0.9946)
(0.4708, 0.9356, 0.9957)
*it=6000 cost=0.6221305727958679
(0.4831, 0.9416, 0.9969)
(0.4888, 0.9425, 0.9969)
*it=7000 cost=0.5543808341026306
(0.5071, 0.9561, 0.997)
(0.5172, 0.9589, 0.998)
*it=8000 cost=0.576149582862854
(0.4935, 0.9532, 0.9975)
(0.4952, 0.9491, 0.9977)
*it=9000 cost=0.5163470506668091
(0.5239, 0.9611, 0.9979)
(0.5313, 0.9646, 0.9985)
*it=10000 cost=0.5203424096107483
(0.5294, 0.9601, 0.999)
(0.5294, 0.9587, 0.9977)
*it=11000 cost=0.4879729449748993
(0.5408, 0.9658, 0.9982)
(0.5368, 0.9683, 0.9982)
*it=12000 cost=0.4

*it=98000 cost=0.2829531133174896
(0.6674, 0.9919, 0.9999)
(0.6653, 0.9924, 0.9999)
*it=99000 cost=0.2872459590435028
(0.6673, 0.9918, 0.9997)
(0.6717, 0.9907, 0.9994)
*it=100000 cost=0.2863863408565521
(0.657, 0.9922, 0.9998)
(0.6648, 0.9921, 0.9998)
*it=101000 cost=0.3031518757343292
(0.6396, 0.9916, 0.9999)
(0.6524, 0.9912, 0.9996)
*it=102000 cost=0.2816827595233917
(0.6631, 0.9919, 0.9998)
(0.6706, 0.9905, 0.9996)
*it=103000 cost=0.27986449003219604
(0.6703, 0.9916, 0.9999)
(0.6587, 0.9926, 0.9995)
*it=104000 cost=0.2812266945838928
(0.6682, 0.9922, 0.9997)
(0.6726, 0.9928, 0.9998)
*it=105000 cost=0.2766740024089813
(0.6698, 0.9931, 0.9999)
(0.6654, 0.992, 0.9998)
*it=106000 cost=0.2813403308391571
(0.667, 0.9925, 0.9996)
(0.6687, 0.9931, 0.9995)
*it=107000 cost=0.27203917503356934
(0.6743, 0.9926, 0.9999)
(0.6705, 0.995, 0.9998)
*it=108000 cost=0.2754461467266083
(0.67, 0.993, 0.9999)
(0.6739, 0.9924, 0.9996)
*it=109000 cost=0.2926148474216461
(0.6544, 0.9916, 0.9998)
(0.6525, 0.9

*it=195000 cost=0.24868208169937134
(0.7027, 0.9942, 0.9996)
(0.6971, 0.9951, 0.9999)
*it=196000 cost=0.2520337402820587
(0.6942, 0.994, 0.9999)
(0.6917, 0.9964, 0.9998)
*it=197000 cost=0.2463965266942978
(0.6954, 0.9945, 0.9998)
(0.6943, 0.9943, 0.9997)
*it=198000 cost=0.24666926264762878
(0.6965, 0.9959, 0.9998)
(0.6947, 0.9958, 0.9999)
*it=199000 cost=0.2574489712715149
(0.6825, 0.9951, 0.9999)
(0.6896, 0.9938, 0.9998)
*it=200000 cost=0.24240832030773163
(0.7053, 0.995, 0.9999)
(0.6946, 0.9945, 1.0)
*it=201000 cost=0.2518657147884369
(0.6948, 0.9952, 0.9999)
(0.6939, 0.9939, 0.9998)
*it=202000 cost=0.27132877707481384
(0.6776, 0.9925, 0.9998)
(0.6878, 0.9924, 0.9996)
*it=203000 cost=0.242034450173378
(0.7002, 0.9956, 0.9998)
(0.6998, 0.9949, 0.9998)
*it=204000 cost=0.2557981014251709
(0.6928, 0.993, 1.0)
(0.6974, 0.9949, 0.9998)
*it=205000 cost=0.2484382539987564
(0.699, 0.9945, 0.9998)
(0.6888, 0.9933, 0.9996)
*it=206000 cost=0.24508720636367798
(0.701, 0.9954, 1.0)
(0.6921, 0.994,

*it=293000 cost=0.22194761037826538
(0.7192, 0.9967, 0.9999)
(0.7233, 0.9951, 0.9998)
*it=294000 cost=0.23606914281845093
(0.7079, 0.996, 0.9994)
(0.7132, 0.9942, 0.9997)
*it=295000 cost=0.22857344150543213
(0.7111, 0.9965, 0.9997)
(0.7085, 0.9954, 0.9998)
*it=296000 cost=0.23195327818393707
(0.709, 0.9968, 0.9998)
(0.6993, 0.9968, 0.9997)
*it=297000 cost=0.2303430140018463
(0.7065, 0.9965, 0.9999)
(0.7088, 0.9965, 1.0)
*it=298000 cost=0.22584614157676697
(0.7152, 0.9963, 1.0)
(0.724, 0.9964, 0.9999)
*it=299000 cost=0.2307005375623703
(0.7147, 0.9956, 0.9999)
(0.7103, 0.9943, 0.9999)
*it=300000 cost=0.22754636406898499
(0.7192, 0.9955, 0.9998)
(0.7145, 0.9957, 0.9996)
*it=301000 cost=0.23195780813694
(0.7064, 0.9961, 0.9999)
(0.7107, 0.9952, 0.9998)
*it=302000 cost=0.23740971088409424
(0.7065, 0.9956, 0.9998)
(0.6955, 0.9953, 0.9999)
*it=303000 cost=0.22023174166679382
(0.7238, 0.9967, 0.9998)
(0.7208, 0.9953, 0.9999)
*it=304000 cost=0.2232070118188858
(0.7229, 0.9958, 1.0)
(0.7203, 0.

*it=391000 cost=0.21484535932540894
(0.7314, 0.9966, 0.9998)
(0.7319, 0.9966, 0.9999)
*it=392000 cost=0.2190036177635193
(0.7238, 0.9969, 1.0)
(0.7174, 0.9954, 1.0)
*it=393000 cost=0.21875496208667755
(0.7209, 0.9975, 1.0)
(0.7136, 0.9954, 0.9999)
*it=394000 cost=0.2183946818113327
(0.7193, 0.9967, 0.9999)
(0.7267, 0.9967, 0.9998)
*it=395000 cost=0.22015254199504852
(0.724, 0.9954, 0.9998)
(0.7245, 0.9964, 0.9998)
*it=396000 cost=0.21263808012008667
(0.7304, 0.9971, 1.0)
(0.722, 0.9955, 0.9997)
*it=397000 cost=0.22847451269626617
(0.7146, 0.996, 0.9999)
(0.728, 0.9965, 0.9998)
*it=398000 cost=0.21965520083904266
(0.72, 0.997, 1.0)
(0.7125, 0.9963, 0.9999)
*it=399000 cost=0.21600426733493805
(0.7286, 0.9974, 1.0)
(0.7215, 0.9962, 0.9998)
*it=400000 cost=0.21364814043045044
(0.7286, 0.997, 1.0)
(0.7289, 0.9948, 0.9998)
*it=401000 cost=0.21110273897647858
(0.7402, 0.9963, 0.9997)
(0.7271, 0.9956, 0.9996)
*it=402000 cost=0.22737714648246765
(0.7163, 0.9959, 0.9998)
(0.7187, 0.9965, 0.9999)

*it=490000 cost=0.2104916274547577
(0.7334, 0.9974, 0.9999)
(0.7371, 0.9974, 1.0)
*it=491000 cost=0.20815037190914154
(0.7351, 0.9972, 0.9998)
(0.7366, 0.9975, 0.9999)
*it=492000 cost=0.21508671343326569
(0.7301, 0.9965, 1.0)
(0.7228, 0.9962, 1.0)
*it=493000 cost=0.22257260978221893
(0.7263, 0.9971, 0.9998)
(0.7187, 0.996, 0.9995)
*it=494000 cost=0.20975300669670105
(0.7363, 0.9962, 0.9996)
(0.7256, 0.9964, 0.9997)
*it=495000 cost=0.21019341051578522
(0.7316, 0.9972, 1.0)
(0.7343, 0.9979, 0.9999)
*it=496000 cost=0.2139996588230133
(0.7266, 0.9973, 0.9999)
(0.7325, 0.9953, 0.9998)
*it=497000 cost=0.2171967327594757
(0.7308, 0.9962, 1.0)
(0.7277, 0.9962, 0.9998)
*it=498000 cost=0.2120209038257599
(0.7326, 0.9977, 1.0)
(0.7358, 0.9963, 0.9998)
*it=499000 cost=0.207169309258461
(0.741, 0.9976, 0.9999)
(0.7362, 0.9971, 0.9998)
*it=500000 cost=0.20729638636112213
(0.7379, 0.9972, 0.9999)
(0.7371, 0.9969, 0.9996)
*it=501000 cost=0.2097160816192627
(0.7317, 0.9971, 0.9998)
(0.73, 0.9961, 0.999

*it=589000 cost=0.20202994346618652
(0.7409, 0.998, 0.9999)
(0.7375, 0.9969, 0.9999)
*it=590000 cost=0.20364683866500854
(0.7383, 0.9983, 1.0)
(0.739, 0.9967, 1.0)
*it=591000 cost=0.2036197930574417
(0.7417, 0.9974, 1.0)
(0.7404, 0.9964, 0.9998)
*it=592000 cost=0.2056475579738617
(0.7388, 0.9982, 1.0)
(0.7348, 0.9966, 1.0)
*it=593000 cost=0.20779448747634888
(0.7362, 0.9974, 0.9999)
(0.7426, 0.9975, 1.0)
*it=594000 cost=0.204445019364357
(0.7427, 0.9971, 1.0)
(0.7414, 0.9959, 1.0)
*it=595000 cost=0.1928195059299469
(0.7543, 0.9974, 1.0)
(0.7433, 0.9969, 0.9999)
*it=596000 cost=0.19810841977596283
(0.7485, 0.9971, 0.9999)
(0.7358, 0.9972, 0.9997)
*it=597000 cost=0.20652920007705688
(0.7398, 0.9968, 0.9997)
(0.7339, 0.9972, 0.9999)
*it=598000 cost=0.19609719514846802
(0.747, 0.998, 0.9998)
(0.7467, 0.9969, 0.9998)
*it=599000 cost=0.20004650950431824
(0.7439, 0.997, 0.9999)
(0.737, 0.9976, 0.9995)
*it=600000 cost=0.20051559805870056
(0.7424, 0.998, 0.9999)
(0.745, 0.9967, 0.9999)
*it=6010

In [ ]:
import pandas as pd
path = '../../bridgent/preds/z_v3_preds_spades.pickle'
pd.to_pickle({'preds':predictions, 'acc012':acc012(y_test.ravel(), predictions.round().ravel())}, path)

In [15]:
import pandas as pd
path = '../../bridgent/preds/z_v3_preds_spades.pickle'
pd.read_pickle(path)

{'preds': array([[8.830617 , 4.4600153, 6.0081186, ..., 3.9872606, 4.4215193,
         6.2920637]], dtype=float32),
 'acc012': (0.77175, 0.99824, 0.99996)}